### Imports

In [3]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [405]:
%run "./instance_10.ipynb"

In [ ]:
# %run "./instance_20.ipynb"

In [ ]:
# %run "./instance_40.ipynb"

### Optimization Problems

In [406]:
### Master Problem ###

# suppress Gurobi output
environment = gp.Env(empty = True)
environment.setParam("OutputFlag", 0)
environment.start()

# model
master = gp.Model('master', env = environment)

# variables + objective function
z = master.addVars(potential_hubs, capacity_levels, obj = fixed_set_up_costs_with_capacity_level, vtype = GRB.BINARY, name = "z")
eta = master.addVar(1, obj = 1, name = 'eta')

# model sense
master.ModelSense = GRB.MINIMIZE

# constraints
master.addConstr(gp.quicksum(gp.quicksum(new_hub_capacities[i, q] * z[i, q] for i in potential_hubs) for q in capacity_levels) >= gp.quicksum(demand_flattened[k - 1] for k in commodities));
master.addConstrs(gp.quicksum(z[i, q] for q in capacity_levels) <= 1 for i in potential_hubs);

In [407]:
def solve_master():

    # update and solve the model
    master.update()
    master.write('master.lp')
    master.optimize()

    z_hat = {}

    for i in potential_hubs:
        for q in capacity_levels:
            z_hat[(i, q)] = z[i, q].X

    master_opt_val = master.ObjVal

    return z_hat, master_opt_val

In [408]:
def solve_dsp(z_hat):

    # suppress Gurobi output
    environment = gp.Env(empty = True)
    environment.setParam("OutputFlag", 0)
    environment.start()

    # model
    dsp = gp.Model('dsp', env = environment)

    # variables + objective function
    alpha = dsp.addVars(commodities, name = "alpha")
    u = dsp.addVars(potential_hubs, commodities, lb = 0, name = "u")
    v = dsp.addVars(potential_hubs, lb = 0, name = "v")

    obj = gp.quicksum(alpha[k] for k in commodities) - gp.quicksum(z_hat[i, q] * u[i, k] for q in capacity_levels for i in potential_hubs for k in commodities) - gp.quicksum(new_hub_capacities[i, q] * z_hat[i, q] * v[i] for q in capacity_levels for i in potential_hubs)

    dsp.setObjective(obj)

    # model sense
    dsp.ModelSense = GRB.MAXIMIZE

    # constraints
    for i in potential_hubs:
        for j in potential_hubs:
            for k in commodities:

                if i != j:
                    dsp.addConstr(alpha[k] - u[i, k] - u[j, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))
                dsp.addConstr(alpha[k] - u[i, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, i, k))

    # update and solve the model
    dsp.update()
    dsp.write('dsp.lp')
    dsp.optimize()

    alpha_vals = {}
    u_vals = {}
    v_vals = {}

    for i in potential_hubs:
        for k in commodities:
            v_vals[i] = v[i].X
            alpha_vals[k] = alpha[k].X
            u_vals[i, k] = u[i, k].X

    dsp_opt_val = dsp.ObjVal

    return alpha_vals, u_vals, v_vals, dsp_opt_val

### Algorithm

In [409]:
def setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat):

    sum = 0

    for i in potential_hubs:
        for q in capacity_levels:
            sum += fixed_set_up_costs_with_capacity_level[i, q] * z_hat[i, q]

    return sum

In [410]:
def benders_decomp():
    
    upper_bound = np.inf
    t = 0
    dsp_extreme_pts = []
    terminate = False

    # print(f"upper_bound at iteration {t}: {upper_bound}")

    while (terminate == False) & (t <= 1000):
        z_hat, master_opt_val = solve_master()

        if master_opt_val == upper_bound:
            # print(f"Done at {t} steps")
            terminate = True
        else:
            alpha_new, u_new, v_new, dsp_opt_val = solve_dsp(z_hat)
            master.addConstr(eta >= gp.quicksum(alpha_new[k] for k in commodities) 
                                  - gp.quicksum(z[i, q] * u_new[i, k] for q in capacity_levels for i in potential_hubs for k in commodities)
                                  - gp.quicksum(new_hub_capacities[i, q] * z[i, q] * v_new[i] for q in capacity_levels for i in potential_hubs))

            if dsp_opt_val + setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat) < upper_bound:
                upper_bound = dsp_opt_val + setup_cost_from_master(potential_hubs, capacity_levels, fixed_set_up_costs_with_capacity_level, z_hat)

        t += 1
        # print(f"upper_bound at iteration {t}: {upper_bound}")

    return z_hat, upper_bound

In [411]:
z_hat, upper_bound = benders_decomp()

In [412]:
z_hat

{(1, 1): 1.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (2, 1): 1.0,
 (2, 2): 0.0,
 (2, 3): 0.0,
 (3, 1): 1.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (4, 1): 1.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (5, 1): 1.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (6, 1): 1.0,
 (6, 2): 0.0,
 (6, 3): 0.0,
 (7, 1): 1.0,
 (7, 2): 0.0,
 (7, 3): 0.0,
 (8, 1): 1.0,
 (8, 2): 0.0,
 (8, 3): 0.0,
 (9, 1): 1.0,
 (9, 2): 0.0,
 (9, 3): 0.0,
 (10, 1): 1.0,
 (10, 2): 0.0,
 (10, 3): 0.0}

In [4]:
# for hub_capacity_pair in z_hat.keys():
#     print(z_hat{hub_capacity_pair})